In [ ]:
import rich

In [1]:
fpath = 'README.md'
with open(fpath, 'rb') as f:
    text = f.read()
text = text.decode("utf-8") 

In [28]:
sections = []
section = []
for i, line in enumerate(text.split('\n')):
    if line.startswith('# '):
        if section:
            sections.append(section)
        section = []
    #section.append(line[2:])
    section.append(line)

entries = []
for i, sec in enumerate(sections):
    if i < 1:
        continue
    entry = {}
    #entry['heading'], entry['rest'] = sec[0], sec[1:]
    entry['heading'], entry['rest'] = sec[0][2:].strip(), sec[1:]
    entries.append(entry)
    
from collections import defaultdict

subheading = ""
for entry in entries:
    subheadings = defaultdict(list)
    for line in entry['rest']:
        if line.startswith('* '):
            subheading = line.strip()[2:]
            continue
        if subheading:
            line = line.strip()
            if line:
                subheadings[subheading].append(line)
    entry.pop('rest')
    if subheadings:
        entry['subheadings'] = subheadings
        
errors = []
for entry in entries:
    if 'subheadings' not in entry:
        continue # actually I want to remove these, but deal with it later
    for topic, items in entry['subheadings'].items():
        recs = []
        for item in items:
            #if len(item.split('-')) != 3:
            #    print((topic, item))

            rec = {}
            parts = item.split('-')
            rec['year_part'], rec['rest'] = parts[0], parts[1:]
            try:
                rec['year'] = int(rec['year_part'][1:].strip())
                rec.pop('year_part')
            except ValueError:
                errors.append(item)

            parts = '-'.join(rec['rest']).split('](')
            if len(parts) <2:
                continue
            if len(parts) > 2:
                errors.append(item)
                continue
            rec['title_part'], rec['rest'] = parts
            rec['title_part'] = rec['title_part'].strip()[1:] # remove starting bracket. need to standardize/remove quotes
            
            parts = rec['rest'].split(') -') 
            if len(parts) <2:
                continue
            if len(parts) > 2:
                errors.append(item)
                continue
            rec['url'], rec['authors_part'] = parts
            rec.pop('rest')
            
            if rec['url'].endswith('.pdf'):
                rec['is_pdf'] = 'True'
                
            rec['authors'] = rec['authors_part'].strip().split(',')
            rec.pop('authors_part')
            
            recs.append(rec)
            
        if recs:
            entry['subheadings'][topic] = recs

In [29]:
for entry in entries:
    if 'subheadings' not in entry: # yeesh...
        continue
    for recs in entry['subheadings'].values():
        for rec in recs:
            if not isinstance(rec, dict):
                print(rec)
                continue
            if not rec['title_part'].startswith(' ["'):
                print(rec)

{'year': 1996, 'title_part': '"Regression Shrinkage and Selection via the Lasso"', 'url': 'https://statweb.stanford.edu/~tibs/lasso/lasso.pdf', 'is_pdf': 'True', 'authors': ['Robert Tibshirani']}
{'year': 2005, 'title_part': '"Regularization and variable selection via the elastic net"', 'url': 'https://web.stanford.edu/~hastie/Papers/B67.2%20(2005)%20301-320%20Zou%20&%20Hastie.pdf', 'is_pdf': 'True', 'authors': ['Hui Zou', ' Trevor Hastie']}
{'year': 1990, 'title_part': '"The Strength of Weak Learnability"', 'url': 'http://rob.schapire.net/papers/strengthofweak.pdf', 'is_pdf': 'True', 'authors': ['Robert E. Schapire']}
{'year': 1991, 'title_part': '"Bagging Predictors"', 'url': 'https://www.stat.berkeley.edu/~breiman/bagging.pdf', 'is_pdf': 'True', 'authors': ['Leo Breiman']}
{'year': 2001, 'title_part': '"Random Forests"', 'url': 'https://link.springer.com/content/pdf/10.1023/A:1010933404324.pdf', 'is_pdf': 'True', 'authors': ['Leo Breiman']}
{'year': 1997, 'title_part': 'AdaBoost) ["

In [38]:
# We're not quite there yet, but the last step will look something like this:
for entry in entries:
    if 'subheadings' not in entry: # yeesh...
        continue
    for subheading, recs in entry['subheadings'].items():
        for rec in recs:
            if not 'url' in rec:
                continue
            #path = f"{entry['heading']} / {subheading}"
            #print((path, rec))
            meta_rec = {
                'heading':entry['heading'],
                'subheading':subheading,
                **rec
            }
            rich.print(meta_rec)

{
    'heading': '"Classic" ML',
    'subheading': 'Lasso/elasticnet',
    'year': 1996,
    'title_part': '"Regression Shrinkage and Selection via the Lasso"',
    'url': 'https://statweb.stanford.edu/~tibs/lasso/lasso.pdf',
    'is_pdf': 'True',
    'authors': ['Robert Tibshirani']
}

{
    'heading': '"Classic" ML',
    'subheading': 'Lasso/elasticnet',
    'year': 2005,
    'title_part': '"Regularization and variable selection via the elastic net"',
    'url': 
'https://web.stanford.edu/~hastie/Papers/B67.2%20(2005)%20301-320%20Zou%20&%20Hastie.pdf',
    'is_pdf': 'True',
    'authors': ['Hui Zou', ' Trevor Hastie']
}

{
    'heading': '"Classic" ML',
    'subheading': 'Boosting',
    'year': 1990,
    'title_part': '"The Strength of Weak Learnability"',
    'url': 'http://rob.schapire.net/papers/strengthofweak.pdf',
    'is_pdf': 'True',
    'authors': ['Robert E. Schapire']
}

{
    'heading': '"Classic" ML',
    'subheading': 'Bagging',
    'year': 1991,
    'title_part': '"Bagging Predictors"',
    'url': 'https://www.stat.berkeley.edu/~breiman/bagging.pdf',
    'is_pdf': 'True',
    'authors': ['Leo Breiman']
}

{
    'heading': '"Classic" ML',
    'subheading': 'random forest',
    'year': 2001,
    'title_part': '"Random Forests"',
    'url': 'https://link.springer.com/content/pdf/10.1023/A:1010933404324.pdf',
    'is_pdf': 'True',
    'authors': ['Leo Breiman']
}

...

{
    'heading': 'Learning theory / Deep learning theory / model compression / 
interpretability',
    'subheading': 'Universal approximation theorem',
    'year': 1991,
    'title_part': '"Approximation capabilities of multilayer feedforward networks"',
    'url': 'https://web.njit.edu/~usman/courses/cs677_spring21/hornik-nn-1991.pdf',
    'is_pdf': 'True',
    'authors': ['Kurt Hornik']
}

{
    'heading': 'Learning theory / Deep learning theory / model compression / 
interpretability',
    'subheading': 'Universal approximation theorem',
    'year': 1993,
    'title_part': '"Multilayer Feedforward Networks With a Nonpolynomial Activation Function 
Can Approximate Any Function"',
    'url': 
'http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.145.6041&rep=rep1&type=pdf',
    'authors': ['Moshe Leshno', ' Lin Vladimir Ya', ' Allan Pinkus', ' Shimon Schocken']
}

{
    'heading': 'Causal Modeling / experimentation',
    'subheading': 'Double machine learning',
    'year': 2016,
    'title_part': '"Double/Debiased Machine Learning for Treatment and Causal Parameters"',
    'url': 'https://arxiv.org/abs/1608.00060',
    'authors': [
        'Victor Chernozhukov',
        ' Denis Chetverikov',
        ' Mert Demirer',
        ' Esther Duflo',
        ' Christian Hansen',
        ' Whitney Newey',
        ' James Robins'
    ]
}

{
    'heading': 'Time series forecasting',
    'subheading': 'RNN forecasting',
    'year': 1991,
    'title_part': '"Recurrent Networks and NARMA Modeling"',
    'url': 
'https://proceedings.neurips.cc/paper/1991/file/5ef0b4eba35ab2d6180b0bca7e46b6f9-Paper.pdf',
    'is_pdf': 'True',
    'authors': ['J. Connor', ' L. Atlas', ' R. Martin']
}

{
    'heading': 'Time series forecasting',
    'subheading': 'RNN forecasting',
    'year': 2017,
    'title_part': '"A Multi-Horizon Quantile Recurrent Forecaster"',
    'url': 'https://arxiv.org/pdf/1711.11053.pdf',
    'is_pdf': 'True',
    'authors': [
        '(Amazon) Ruofeng Wen',
        ' Kari Torkkola',
        ' Balakrishnan Narayanaswamy',
        ' Dhruv Madeka'
    ]
}

In [36]:
for entry in entries:
    rich.print(entry)

{
    'heading': '"Classic" ML',
    'subheadings': defaultdict(<class 'list'>, {
        'Lasso/elasticnet': [
            {
                'year': 1996,
                'title_part': '"Regression Shrinkage and Selection via the Lasso"',
                'url': 'https://statweb.stanford.edu/~tibs/lasso/lasso.pdf',
                'is_pdf': 'True',
                'authors': ['Robert Tibshirani']
            },
            {
                'year': 2005,
                'title_part': '"Regularization and variable selection via the elastic net"',
                'url': 
'https://web.stanford.edu/~hastie/Papers/B67.2%20(2005)%20301-320%20Zou%20&%20Hastie.pdf',
                'is_pdf': 'True',
                'authors': ['Hui Zou', ' Trevor Hastie']
            }
        ],
        'Boosting': [
            {
                'year': 1990,
                'title_part': '"The Strength of Weak Learnability"',
                'url': 'http://rob.schapire.net/papers/strengthofweak.pdf',
                'is_pdf': 'True',
                'authors': ['Robert E. Schapire']
            }
        ],
        'Bagging': [
            {
                'year': 1991,
                'title_part': '"Bagging Predictors"',
                'url': 'https://www.stat.berkeley.edu/~breiman/bagging.pdf',
                'is_pdf': 'True',
                'authors': ['Leo Breiman']
            }
        ],
        'random forest': [
            {
                'year': 2001,
                'title_part': '"Random Forests"',
                'url': 'https://link.springer.com/content/pdf/10.1023/A:1010933404324.pdf',
                'is_pdf': 'True',
                'authors': ['Leo Breiman']
            }
        ],
        'Adaboost': [
            {
                'year': 1997,
                'title_part': 'AdaBoost) ["A Decision-Theoretic Generalization of On-Line 
Learning and an Application to Boosting"',
                'url': 'https://www.sciencedirect.com/science/article/pii/S002200009791504X',
                'authors': ['Yoav Freund', ' Robert E Schapire']
            },
            {
                'year': 1999,
                'title_part': '"Improved Boosting Algorithms Using Confidence-rated 
Predictions"',
                'url': 'https://sci2s.ugr.es/keel/pdf/algorithm/articulo/1999-ML-Improved%20b
oosting%20algorithms%20using%20confidence-rated%20predictions%20(Schapire%20y%20Singer\\\\).p
df',
                'is_pdf': 'True',
                'authors': ['Yoav Freund', ' Robert E Schapire']
            }
        ],
        'gradient boosting': [
            {
                'year': 2016,
                'title_part': '"XGBoost: A Scalable Tree Boosting System"',
                'url': 'https://arxiv.org/pdf/1603.02754.pdf',
                'is_pdf': 'True',
                'authors': ['Tianqi Chen', ' Carlos Guestrin']
            }
        ],
        'bias-variance tradeoff': [
            {
                'year': 1997,
                'title_part': '"Bias Plus Variance Decomposition for Zero-One Loss 
Functions"',
                'url': 'https://www.researchgate.net/publication/2793430_Bias_Plus_Variance_D
ecomposition_for_Zero-One_Loss_Functions',
                'authors': ['Ron Kohavi', ' David H. Wolpert']
            }
        ],
        'non-parametric bootstrap': [
            {
                'year': 1979,
                'title_part': '"Bootstrap Methods: Another Look at the Jackknife"',
                'url': 'https://projecteuclid.org/journals/annals-of-statistics/volume-7/issu
e-1/Bootstrap-Methods-Another-Look-at-the-Jackknife/10.1214/aos/1176344552.full',
                'authors': ['Bradley Effron']
            }
        ],
        'PCA': [
            {
                'year': 1991,
                'title_part': '"Face Recognition Using Eigenfaces"',
                'url': 
'https://www.cin.ufpe.br/~rps/Artigos/Face%20Recognition%20Using%20Eigenfaces.pdf',
                'is_pdf': 'True',
                'authors':

{
    'heading': 'Network Graphs / combinatorial optimization',
    'subheadings': defaultdict(<class 'list'>, {
        'modularity / louvain community detection': [
            {
                'year': 2004,
                'title_part': '"Finding community structure in very large networks"',
                'url': 'https://arxiv.org/abs/cond-mat/0408187',
                'authors': ['Aaron Clauset', ' M. E. J. Newman', ' Cristopher Moore']
            },
            {
                'year': 2008,
                'title_part': '"Fast unfolding of communities in large networks"',
                'url': 'https://arxiv.org/abs/0803.0476',
                'authors': [
                    'Vincent D. Blondel',
                    ' Jean-Loup Guillaume',
                    ' Renaud Lambiotte',
                    ' Etienne Lefebvre'
                ]
            }
        ],
        'pagerank': [
            {
                'year': 1998,
                'title_part': '"The PageRank Citation Ranking: Bringing Order to the Web"',
                'url': 'http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf',
                'is_pdf': 'True',
                'authors': ['Larry Page']
            }
        ],
        'label propagation': [
            {
                'year': 2002,
                'title_part': '"Learning From Labeled and Unlabeled Data With Label 
Propagation"',
                'url': 'http://mlg.eng.cam.ac.uk/zoubin/papers/CMU-CALD-02-107.pdf',
                'is_pdf': 'True',
                'authors': ['Xiaojin Zhu', ' Zoubin Ghahramani']
            }
        ]
    })
}

{
    'heading': 'Misc optimization and numerical methods',
    'subheadings': defaultdict(<class 'list'>, {
        'Constraint Programming / queing theory / OR': [
            '* Uh... here there be dragons. Maybe just leave some breadcrumbs here?'
        ]
    })
}

{
    'heading': 'Neural optimizers',
    'subheadings': defaultdict(<class 'list'>, {
        'SGD / backprop': [
            {
                'year': 1986,
                'title_part': '"Learning representations by back-propagating errors"',
                'url': 'http://www.cs.utoronto.ca/~hinton/absps/naturebp.pdf',
                'is_pdf': 'True',
                'authors': ['David Rumelhart', ' Geoffrey Hinton', ' Ronald Williams']
            },
            {
                'year': 1998,
                'title_part': '"Efficient Backprop"',
                'url': 'http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Yann LeCun',
                    ' Leon Bottou',
                    ' Genevieve B. Orr and Klaus-Robert Müller'
                ]
            }
        ],
        'Adagrad / RMSProp': ['* Probably discussed sufficiently in the Adam paper'],
        'Adam': [
            {
                'year': 2014,
                'title_part': '"Adam: A Method for Stochastic Optimization"',
                'url': 'https://arxiv.org/abs/1412.6980',
                'authors': ['Diederik P. Kingma', ' Jimmy Ba']
            }
        ],
        'reverse-mode autodiff': ['* see backprop'],
        'distributed training': [
            {
                'year': 2011,
                'title_part': '"HOGWILD!: A Lock-Free Approach to Parallelizing Stochastic 
Gradient Descent"',
                'url': 'https://people.eecs.berkeley.edu/~brecht/papers/hogwildTR.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Feng Niu',
                    ' Benjamin Recht',
                    ' Christopher Re',
                    ' Stephen J. Wright'
                ]
            }
        ]
    })
}

{
    'heading': 'Neural activations',
    'subheadings': defaultdict(<class 'list'>, {
        'ReLU': [
            {
                'year': 2011,
                'title_part': '"Deep Sparse Rectifier Neural Networks"',
                'url': 'http://proceedings.mlr.press/v15/glorot11a/glorot11a.pdf',
                'is_pdf': 'True',
                'authors': ['Xavier Glorot', '  Antoine Bordes', ' Yoshua Bengio']
            }
        ]
    })
}

...

{
    'heading': 'Specific architectures/achievements, and other misc milestones',
    'subheadings': defaultdict(<class 'list'>, {
        'LeNet': [
            {
                'year': 1998,
                'title_part': '"GradientBased Learning Applied to Document Recognition"',
                'url': 'http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Yann LeCun',
                    ' Leon Bottou',
                    ' Yoshua Bengio',
                    ' Patrick Haffner'
                ]
            }
        ],
        'alexnet - Demonstrated importance of network depth (specifically stacking 
convolutions), and ReLU capability over the then conventional sigmoid and tanh activations': 
[
            {
                'year': 2012,
                'title_part': 'using ImageNet leaderboard date; article published 2017) - 
["ImageNet Classification with Deep Convolutional Neural Networks"',
                'url': 
'https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf',
                'is_pdf': 'True',
                'authors': ['Alex Krizhevsky', ' Ilya Sutskever', ' Geoffrey E. Hinton']
            }
        ],
        'BERT': [
            {
                'year': 2018,
                'title_part': '"BERT: Pre-training of Deep Bidirectional Transformers for 
Language Understanding"',
                'url': 'https://arxiv.org/abs/1810.04805v2',
                'authors': [
                    'Jacob Devlin',
                    ' Ming-Wei Chang',
                    ' Kenton Lee',
                    ' Kristina Toutanova'
                ]
            }
        ],
        'RNN-LM': [
            {
                'year': 2010,
                'title_part': '"Recurrent neural network based language model"',
                'url': 
'https://www.isca-speech.org/archive/archive_papers/interspeech_2010/i10_1045.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Tomas Mikolov',
                    ' Martin Karafiat',
                    " Luka's Burget",
                    ' Jan "Honza" Cernocky',
                    ' Sanjeev Khudanpur'
                ]
            },
            {
                'year': 2014,
                'title_part': '"Generating Sequences With Recurrent Neural Networks"',
                'url': 'https://arxiv.org/pdf/1308.0850.pdf',
                'is_pdf': 'True',
                'authors': ['Alex Graves']
            }
        ],
        'word2vec': [
            {
                'year': 2013,
                'title_part': '"Distributed Representations of Words and Phrases and their 
Compositionality"',
                'url': 
'https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Tomas Mikolov',
                    ' Ilya Sutskever',
                    ' Kai Chen',
                    ' Greg Corrado',
                    ' Jeffrey Dean'
                ]
            }
        ],
        'siamese network': [
            {
                'year': 2015,
                'title_part': '"Siamese Neural Networks for One-Shot Image Recognition"',
                'url': 'http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf',
                'is_pdf': 'True',
                'authors': ['Gregory Koch']
            }
        ],
        'student-teacher transfer learning, catastrophic forgetting': [
            '* see also knowledge distillation below'
        ],
        'VGG': [
            {
                'year': 2014,
                'title_part': '"Very Deep Convolutional Networks for Large-Scale Image 
Recognition"',
                'url': 'https://arxiv.org/abs/1409.1556',
                'authors': ['Karen Simonyan', ' Andrew Zisserman']
            }
        ],
        'inception/DeepDream': [
            

{
    'heading': 'Learning theory / Deep learning theory / model compression / 
interpretability',
    'subheadings': defaultdict(<class 'list'>, {
        'VC Dimension': [
            {
                'year': 1971,
                'title_part': '"On the uniform convergence of relative frequencies of events 
to their probabilities"',
                'url': 'https://courses.engr.illinois.edu/ece544na/fa2014/vapnik71.pdf',
                'is_pdf': 'True',
                'authors': ['V. Vapnik and A. Chervonenkis']
            },
            {
                'year': 1989,
                'title_part': '"Learnability and the Vapnik-Chervonenkis Dimension "',
                'url': 'https://www.trhvidsten.com/docs/classics/Blumer-1989.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Blumer',
                    ' A.; Ehrenfeucht',
                    ' A.; Haussler',
                    ' D.; Warmuth',
                    ' M. K.'
                ]
            }
        ],
        'gradient double descent': [
            {
                'year': 2019,
                'title_part': '"Deep Double Descent: Where Bigger Models and More Data 
Hurt"',
                'url': 'https://arxiv.org/abs/1912.02292',
                'authors': [
                    'Preetum Nakkiran',
                    ' Gal Kaplun',
                    ' Yamini Bansal',
                    ' Tristan Yang',
                    ' Boaz Barak',
                    ' Ilya Sutskever'
                ]
            }
        ],
        'neural tangent kernel': [
            {
                'year': 2018,
                'title_part': '"Neural Tangent Kernel: Convergence and Generalization in 
Neural Networks"',
                'url': 'https://arxiv.org/abs/1806.07572',
                'authors': ['Arthur Jacot', ' Franck Gabriel', ' Clément Hongler']
            }
        ],
        'lottery ticket hypothesis': [
            {
                'year': 2018,
                'title_part': '"The Lottery Ticket Hypothesis: Finding Sparse, Trainable 
Neural Networks"',
                'url': 'https://arxiv.org/abs/1803.03635',
                'authors': ['Jonathan Frankle', ' Michael Carbin']
            }
        ],
        'generalized degrees of freedom': [
            '* 1998 - "On Measuring and Correcting the Effects of Data Mining and Model 
Selection" - Jianming Ye'
        ],
        'dropout as ensemblification': [
            {
                'year': 2013,
                'title_part': '"Understanding Dropout"',
                'url': 
'https://papers.nips.cc/paper/2013/file/71f6278d140af599e06ad9bf1ba03cb0-Paper.pdf',
                'is_pdf': 'True',
                'authors': ['Pierre Baldi', ' Peter Sadowski']
            },
            {
                'year': 2017,
                'title_part': '"Analysis of dropout learning regarded as ensemble learning"',
                'url': 'https://arxiv.org/pdf/1706.06859.pdf',
                'is_pdf': 'True',
                'authors': ['Kazuyuki Hara', ' Daisuke Saitoh', ' Hayaru Shouno']
            }
        ],
        'knowledge distillation': [
            {
                'year': 2005,
                'title_part': '"Model Compression"',
                'url': 'http://www.cs.cornell.edu/~caruana/compression.kdd06.pdf',
                'is_pdf': 'True',
                'authors': [
                    'Cristian Bucila',
                    ' Rich Caruana',
                    ' Alexandru Niculescu-Mizil'
                ]
            },
            {
                'year': 2015,
                'title_part': '"Distilling the Knowledge in a Neural Network"',
                'url': 'https://arxiv.org/pdf/1503.02531.pdf',
                'is_pdf': 'True',
                'authors': ['Geoffrey Hinton', ' Oriol Vinyals', ' Jeff Dean']
            }
        ],
        'SGD = MAP inference': [
            {
                'year': 2017,
                'title_part'

{'heading': 'Information theory'}

{
    'heading': 'Causal Modeling / experimentation',
    'subheadings': defaultdict(<class 'list'>, {
        'Double machine learning': [
            {
                'year': 2016,
                'title_part': '"Double/Debiased Machine Learning for Treatment and Causal 
Parameters"',
                'url': 'https://arxiv.org/abs/1608.00060',
                'authors': [
                    'Victor Chernozhukov',
                    ' Denis Chetverikov',
                    ' Mert Demirer',
                    ' Esther Duflo',
                    ' Christian Hansen',
                    ' Whitney Newey',
                    ' James Robins'
                ]
            }
        ]
    })
}

{
    'heading': 'Time series forecasting',
    'subheadings': defaultdict(<class 'list'>, {
        'RNN forecasting': [
            {
                'year': 1991,
                'title_part': '"Recurrent Networks and NARMA Modeling"',
                'url': 
'https://proceedings.neurips.cc/paper/1991/file/5ef0b4eba35ab2d6180b0bca7e46b6f9-Paper.pdf',
                'is_pdf': 'True',
                'authors': ['J. Connor', ' L. Atlas', ' R. Martin']
            },
            {
                'year': 2017,
                'title_part': '"A Multi-Horizon Quantile Recurrent Forecaster"',
                'url': 'https://arxiv.org/pdf/1711.11053.pdf',
                'is_pdf': 'True',
                'authors': [
                    '(Amazon) Ruofeng Wen',
                    ' Kari Torkkola',
                    ' Balakrishnan Narayanaswamy',
                    ' Dhruv Madeka'
                ]
            }
        ]
    })
}

In [31]:
errors

['* Uh... here there be dragons. Maybe just leave some breadcrumbs here?',
 '* Probably discussed sufficiently in the Adam paper',
 '* see backprop',
 '* See also AlexNet',
 '* see also Alex Graves 2013',
 '* see also knowledge distillation below',
 '* http://karpathy.github.io/2019/04/25/recipe/']